
## Autograded Notebook (Canvas & CodeGrade)

This notebook will be automatically graded. It is designed to test your answers and award points for the correct answers. Following the instructions for each Task carefully.
Instructions

- **Download** this notebook as you would any other ipynb file 
- **Upload** to Google Colab or work locally (if you have that set-up)
- **Delete** `raise NotImplementedError()`

- **Write** your code in the `# YOUR CODE HERE` space


- **Execute** the Test cells that contain assert statements - these help you check your work (others contain hidden tests that will be checked when you submit through Canvas)

- **Save** your notebook when you are finished
- **Download** as a ipynb file (if working in Colab)
- **Upload** your complete notebook to Canvas (there will be additional instructions in Slack and/or Canvas)



# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more manageable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
Successfully complete all these objectives to earn full credit. 

**Successful completion is defined as passing all the unit tests in each objective.**  

Each unit test that you pass is 1 point. 

There are 5 total possible points in this sprint challenge. 


There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

____

# Before you submit your notebook you must first

1) Restart your notebook's Kernel

2) Run all cells sequentially, from top to bottom, so that cell numbers are sequential numbers (i.e. 1,2,3,4,5...)
- Easiest way to do this is to click on the **Cell** tab at the top of your notebook and select **Run All** from the drop down menu. 

3) Comment out the cell that generates a pyLDAvis visual in objective 4 (see instructions in that section). 
____



### Import Data

In [ ]:
import pandas as pd

# Load reviews from URL
data_url = 'https://raw.githubusercontent.com/LambdaSchool/data-science-practice-datasets/main/unit_4/unit1_nlp/review_sample.json'

# Import data into a DataFrame named df
df = pd.read_json(data_url, lines=True)

In [ ]:
df.head(5)

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


In [ ]:
# Visible Testing
assert isinstance(df, pd.DataFrame), 'df is not a DataFrame. Did you import the data into df?'
assert df.shape[0] == 10000, 'DataFrame df has the wrong number of rows.'

## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [ ]:
# Optional: Consider using spaCy in your function. The spaCy library can be imported by running this cell.
# A pre-trained model (en_core_web_sm) has been made available to you in the CodeGrade container.
# If you DON'T need use the en_core_web_sm model, you can comment it out below.
import spacy
nlp = spacy.load('en_core_web_sm')

In [ ]:
# Tokenize documents
def tokenize(doc):
  """
  Function takes a single document and creates
  a list of all the tokens in that document
  """
  doc_tokens = []
  for token in nlp(doc):
    if (token.is_stop == False) and (token.is_punct == False):
      doc_tokens.append(token.text.lower())
  return doc_tokens

In [ ]:
'''Testing'''
assert isinstance(tokenize(df.sample(n=1)["text"].iloc[0]), list), "Make sure your tokenizer function accepts a single document and returns a list of tokens!"

In [ ]:
!pip install pandarallel

In [ ]:
from pandarallel import pandarallel

In [ ]:
pandarallel.initialize(progress_bar=True, nb_workers=4)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [ ]:
# Create new feature 'tokens' that is the result of applying the tokenize
# function to the 'text' feature. Parallel apply for efficiency
df['tokens'] = df['text'].parallel_apply(tokenize)

Streaming output truncated to the last 5000 lines.
  25.96% ::::::::::                               |      649 /     2500 |      
  26.16% ::::::::::                               |      654 /     2500 |      MMM  25.08% ::::::::::                               |      627 /     2500 |      
  25.48% ::::::::::                               |      637 /     2500 |      
  26.00% ::::::::::                               |      650 /     2500 |      
  26.20% ::::::::::                               |      655 /     2500 |      MMM  25.12% ::::::::::                               |      628 /     2500 |      
  25.52% ::::::::::                               |      638 /     2500 |      
  26.04% ::::::::::                               |      651 /     2500 |      
  26.24% ::::::::::                               |      656 

In [ ]:
# Check out new feature!
df.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id,tokens
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w,"[beware, fake, fake, fake, small, business, lo..."
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA,"[came, lunch, togo, service, quick, staff, fri..."
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA,"[vegas, dozens, times, stepped, foot, circus, ..."
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ,"[went, night, closed, street, party, best, act..."
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA,"[3.5, 4, stars, \n\n, bad, price, $, 12.99, lu..."


In [ ]:
def list_to_tokens(tokens):
  """
  Converts list of tokens to one long string
  """
  str = " "
  return (str.join(tokens))

df['tokens_string'] = df['tokens'].apply(list_to_tokens)


In [ ]:
df.tokens_string

0       beware fake fake fake small business los alami...
1       came lunch togo service quick staff friendly c...
2       vegas dozens times stepped foot circus circus ...
3       went night closed street party best actually g...
4       3.5 4 stars \n\n bad price $ 12.99 lunch senio...
                              ...                        
9995    family hungry subway open 24 hours guy working...
9996    wife came couple friends sever excited point o...
9997    food ok brag food hot items tasty horrible loo...
9998    today visit great love enjoy town square las v...
9999    absolute worst place stayed 43 year life   tim...
Name: tokens_string, Length: 10000, dtype: object

In [ ]:
# Set X and Y
X = df['text']
X_clean = df['tokens_string']
Y = df['stars']

In [ ]:
# Check out our new tokens_string!
df.head(5)

,business_id,cool,date,funny,review_id,stars,text,useful,user_id,tokens,tokens_string
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w,"[beware, fake, fake, fake, small, business, lo...",beware fake fake fake small business los alami...
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA,"[came, lunch, togo, service, quick, staff, fri...",came lunch togo service quick staff friendly c...
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA,"[vegas, dozens, times, stepped, foot, circus, ...",vegas dozens times stepped foot circus circus ...
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ,"[went, night, closed, street, party, best, act...",went night closed street party best actually g...
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA,"[3.5, 4, stars, \n\n, bad, price, $, 12.99, lu...",3.5 4 stars \n\n bad price $ 12.99 lunch senio...


In [ ]:
# Let's compare our raw text data to our clean text data
X.iloc[0]

'BEWARE!!! FAKE, FAKE, FAKE....We also own a small business in Los Alamitos, CA and received what looked like a legitimate bill for $70 with an account number and all.  I called the phone number listed (866) 273-7934.  The wait time on hold said 20 minutes and to leave a message.  I could not get a live person on the phone no matter what number I selected.  I left a very FIRM message that I would be contacting the BBB and my attorney regarding their company trying to scam businesses. This has to be illegal!!!!!'

In [ ]:
X_clean.iloc[0]

'beware fake fake fake small business los alamitos received looked like legitimate bill $ 70 account number   called phone number listed 866 273 7934   wait time hold said 20 minutes leave message   live person phone matter number selected   left firm message contacting bbb attorney company trying scam businesses illegal'

## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews (i.e. create a doc-term matrix).
2. Write a fake review and query for the 10 most similar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, use `NearestNeighbors` model for this. 

In [ ]:
# Create fake Yelp review for analysis
fake_review = ["I loved this Korean BBQ restaurant! Staff was friendly and portions were phenomenal!"]

In [ ]:
# Create a new dataframe that contains `tokens_list` from original DataFrame
df_new = pd.DataFrame(X_clean)
df_new.head()

,tokens_string
0,beware fake fake fake small business los alami...
1,came lunch togo service quick staff friendly c...
2,vegas dozens times stepped foot circus circus ...
3,went night closed street party best actually g...
4,3.5 4 stars \n\n bad price $ 12.99 lunch senio...


In [ ]:
# Add our fake review to this new DataFrame for vectorization
df_new.loc[len(df_new.index)] = fake_review

In [ ]:
%%time
# Create a vector representation of the reviews 
# Name that doc-term matrix "dtm"

from sklearn.feature_extraction.text import TfidfVectorizer

def vectorizer(data):
  """
  Vectorize the data using TfidfVectorizer. 

  Input: (data) - a Pandas Series containing tokens of each document
  Output: (dtm_df) - Document Term Matrix in a DataFrame to fit to Nearest Neighbors
  """
  tfidf_vect = TfidfVectorizer(stop_words='english', max_features=10000, min_df=5)
  tfidf_vect.fit(data)
  dtm = tfidf_vect.transform(data)
  dtm_df = pd.DataFrame(data=dtm.toarray(), columns=tfidf_vect.get_feature_names())
  return dtm_df

dtm_df = vectorizer(df_new['tokens_string'])

CPU times: user 1.32 s, sys: 423 ms, total: 1.74 s
Wall time: 1.62 s


In [ ]:
dtm_df.head()

,00,000,00am,00pm,05,06,07,10,100,1000,101,10pm,10th,11,110,11am,11pm,12,120,125,13,14,140,145,15,150,1500,16,160,16th,17,170,18,180,19,1st,20,200,2000,2004,...,yelled,yelling,yellow,yellowtail,yelp,yelped,yelper,yelpers,yep,yes,yesterday,yikes,yo,yoga,yogurt,yolk,yolks,yonge,york,yorkville,young,younger,youtube,yr,yuck,yum,yummm,yummy,yup,zach,zen,zero,zest,zing,zip,zone,zoo,zucchini,était,été
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.09768,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.08297,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.140416,0.0,0.0,0.0,0.0,0.0,0.0,0.126699,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Create and fit a NearestNeighbors model named "nn"
from sklearn.neighbors import NearestNeighbors

# Fit NearestNeighbors model to dtm_df
nn = NearestNeighbors(n_neighbors=10, algorithm='kd_tree')
nn.fit(dtm_df)

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                 radius=1.0)

In [ ]:
'''Testing.'''
assert nn.__module__ == 'sklearn.neighbors._unsupervised', ' nn is not a NearestNeighbors instance.'
assert nn.n_neighbors == 10, 'nn has the wrong value for n_neighbors'

**Instructor confirmed that unit test needs to be updated**

In [ ]:
# Make sure our fake review is in df_new, as well as find its index
df_new.tail()

,tokens_string
9996,wife came couple friends sever excited point o...
9997,food ok brag food hot items tasty horrible loo...
9998,today visit great love enjoy town square las v...
9999,absolute worst place stayed 43 year life tim...
10000,I loved this Korean BBQ restaurant! Staff was ...


In [ ]:
df_new.iloc[10000]

tokens_string    I loved this Korean BBQ restaurant! Staff was ...
Name: 10000, dtype: object

In [ ]:
# Data has been vectorized, NN model has been fit, now to find nearest neighbors
doc_index = 10000
doc = [dtm_df.iloc[doc_index].values]

neigh_dist, neigh_index = nn.kneighbors(doc)

In [ ]:
neigh_dist

array([[0.        , 1.        , 1.        , 1.08611517, 1.14977298,
        1.15129766, 1.16585663, 1.16640913, 1.17158578, 1.18151112]])

In [ ]:
similar_reviews = neigh_index
similar_reviews

array([[10000,  6311,  6204,  6090,  7327,  5509,  9232,   238,  5254,
         8223]])

In [ ]:
# Iterate thru similar_reviews and print out the review so we can see if they're similar
# to our fake review!
for doc in similar_reviews:
  print(df_new.iloc[doc])

# Some patterns: These reviews mention korean food, korean bbq, and some korean names! Woohooooo!!

                                           tokens_string
10000  I loved this Korean BBQ restaurant! Staff was ...
6311   天氣很熱吃不下東西，今天我點了一個韓國冷面湯、餐後點了甜點，冰沙系列不會太甜膩，覺得店家很用...
6204   旅行でラスベガスに来ましたがネイルがはげてるのが気になり、探したお店でした \n 質問にも丁...
6090   hit place time come las vegas ventilation syst...
7327   time woonam jung atmosphere nice service frien...
5509   love going gen korean bbq disappointed ordered...
9232   time green chili pork burritio   phenomenal   ...
238    came visit toronto local friend said going opt...
5254   love korean food place unfortunately impress \...
8223   korean bbq restaurants las vegas u.s. friendly...


## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a pipeline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier.
    - Use that pipeline to train a model to predict the `stars` feature (i.e. the labels). 
    - Use that Pipeline to predict a star rating for your fake review from Part 2. 



2. Create a parameter dict including `one parameter for the vectorizer` and `one parameter for the model`. 
    - Include 2 possible values for each parameter
    - **Use `n_jobs` = 1** 
    - Due to limited computational resources on CodeGrader `DO NOT INCLUDE ADDITIONAL PARAMETERS OR VALUES PLEASE.`
    
    
3. Train the entire pipeline with a GridSearch
    - Name your GridSearch object as `gs`

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

# Name the gridsearch instance "gs"

# Create a Pipeline with TfidfVectorizer as our vectorizer,
# and KNeighborsClassifier as our classifier.
tfidf = TfidfVectorizer()
knc = KNeighborsClassifier(n_jobs=1)

pipe = Pipeline([('vect', tfidf),
                 ('clf', knc)])

# Change these hyperparameters in our vectorizer and classifier
params = {
    "vect__max_df": [1.0, 0.90],
    "clf__weights": ['uniform', 'distance']
}

# GridSearchCV to find the best hyperparameters
gs = GridSearchCV(pipe,
                  params,
                  n_jobs=-2,
                  cv=3,
                  verbose=2)

# Fit the model to our data
gs.fit(X_clean, Y)

In [ ]:
# Grab the best model from GridSearchCV
best_model = gs.best_estimator_
best_model

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                      metric='minkowski', metric_params=None,


In [ ]:
# Locate my fake review from DataFrame, predict rating.
my_review = df_new.iloc[10000]
best_model.predict(my_review)

# My review is a 5 star rating! (As expected)

array([5])

In [ ]:
# Visible Testing
prediction = gs.predict(["I wish dogs knew how to speak English."])[0]
assert prediction in df.stars.values, 'You gs object should be able to accept raw text within a list. Did you include a vectorizer in your pipeline?'

## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Set num_topics to `5`
    - Name your LDA model `lda`
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

When you instantiate your LDA model, it should look like this: 

```python
lda = LdaModel(corpus=corpus,
               id2word=id2word,
               random_state=723812,
               num_topics = num_topics,
               passes=1
              )

```

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

## Note about  pyLDAvis

**pyLDAvis** is the Topic modeling package that we used in class to visualize the topics that LDA generates for us.

You are welcomed to use pyLDAvis if you'd like for your visualization. However, **you MUST comment out the code that imports the package and the cell that generates the visualization before you submit your notebook to CodeGrade.** 

Although you should leave the print out of the visualization for graders to see (i.e. comment out the cell after you run it to create the viz). 

In [ ]:
from gensim import corpora
# Due to limited computationalresources on CodeGrader, use the non-multicore version of LDA 
from gensim.models.ldamodel import LdaModel
import gensim
import re

### 1. Estimate a LDA topic model of the review tex

In [ ]:
# Remember to read the LDA docs for more information on the various class attirbutes and methods available to you
# in the LDA model: https://radimrehurek.com/gensim/models/ldamodel.html

# don't change this value 
num_topics = 5

# use tokenize function you created earlier to create tokens  - DONE

# create a id2word object (hint: use corpora.Dictionary) - DONE

# create a corpus object (hint: id2word.doc2bow) - DONE

# instantiate an lda model - DONE

# YOUR CODE HERE

# We're using df instead of df_new because we want to analyze the Yelp reviews (excluding my fake review)
id2word = corpora.Dictionary(df['tokens'])
corpus = [id2word.doc2bow(text) for text in df['tokens']]

# Instantiate our LdaModel
lda = LdaModel(corpus=corpus,
               id2word=id2word,
               random_state=42,
               num_topics=5,
               passes=1)

#### Testing

In [ ]:
# Visible Testing
assert lda.get_topics().shape[0] == 5, 'Did your model complete its training? Did you set num_topics to 5?'

#### 2. Create 1-2 visualizations of the results

In [ ]:
!pip install pyLDAvis

In [ ]:
# COMMENT OUT THIS CELL AFTER RUNNING

import seaborn as sns
import matplotlib.pyplot as plt
import pyLDAvis
import pyLDAvis.gensim_models

# Use pyLDAvis to visualize our results 
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda, corpus, id2word)
vis

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.007700  0.039544       1        1  28.189468
3     -0.079412 -0.055890       2        1  26.318168
1      0.019165  0.089117       3        1  24.565806
2     -0.038836 -0.013246       4        1  16.319712
4      0.106783 -0.059525       5        1   4.606846, topic_info=       Term          Freq         Total Category  logprob  loglift
0            13555.000000  13555.000000  Default  30.0000  30.0000
57     \n\n   8054.000000   8054.000000  Default  29.0000  29.0000
156    food   5066.000000   5066.000000  Default  28.0000  28.0000
354      \n   3130.000000   3130.000000  Default  27.0000  27.0000
223   great   4108.000000   4108.000000  Default  26.0000  26.0000
...     ...           ...           ...      ...      ...      ...
777    love     56.337945   1650.957789   Topic5  -6.0544  -0.3001
988   clean     47.589637    678.755347   Topic5  -6.2232   0.4200
44     came     50.528065   1469.410039   Topic5  -6.1633  -0.2925
1125    new     48.971772   1056.391074   Topic5  -6.1946   0.0063
25     like     51.176382   3365.035647   Topic5  -6.1505  -1.1083

[467 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
354        1  0.062297    \n
354        2  0.413077    \n
354        3  0.235450    \n
354        4  0.220755    \n
354        5  0.068047    \n
...      ...       ...   ...
10949      4  0.934717  yuck
14038      3  0.025595   yuk
14038      5  0.947002   yuk
4212       2  0.027852     à
4212       5  0.974807     à

[1128 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 2, 3, 5])

#### 3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

To be honest, the 5 topics that I was able to find were all pretty similar. Given that I was working in Colab with limited computing power, I was not able to clean my data adequately. I created a copy of this notebook and tried cleaning the data even further using lemmatization and cleaning whitespaces, but Colab crashed and I wasn't able to get better analysis. Next time I will move to Google Cloud.

The 10 similar reviews were actually very similar to my fake review. I mentioned Korean BBQ, and most of the similar reviews mentioned Korean BBQ and all things related to Korea. I wrote am excellent review and my model predicted a review of 5! Success!

For my topic model, all the topics mentioned "good" so there is no clear delineation of topics. I would assume the topics would be spread out by rating, where one of the topics have clear positive buzzwords and the opposite would have clearly negative buzzwords. This is something to look into, and could be accomplished with thorough data cleaning.